In [205]:
from boilerplate import setup_nb
%load_ext autoreload
%autoreload 2
device = setup_nb()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA is available.
Changed working dir to /home/spandan/Projects/hubbard-transformer


In [206]:
from embedding import HubbardEmbedding
import torch
import einops as ein

In [207]:
N_PARAMS = 5
TOKEN_DIMS = 32
PARAM_DIMS = 16
MAX_LEN = 100
MIN_OCC = 0
MAX_OCC = 10
BATCH_SIZE = 32
model_dims = PARAM_DIMS + TOKEN_DIMS

In [208]:
embedding = HubbardEmbedding(TOKEN_DIMS, PARAM_DIMS, N_PARAMS, MAX_LEN).to(device)

In [209]:
(test_params := torch.randn(N_PARAMS, BATCH_SIZE).to(device))

tensor([[-0.2479, -0.0790, -0.5563,  1.0193, -1.5223,  1.0212,  1.1037, -1.4205,
         -0.6462, -1.6686,  1.1274, -0.7758,  0.3754,  0.3069, -2.0173, -1.0540,
          1.0899,  0.6888,  0.1397, -0.2177,  0.6077,  0.7090, -0.8927, -0.4520,
         -1.2204, -1.5151, -0.7829,  1.8253, -0.5316,  1.6677,  0.4271, -0.2675],
        [ 0.3799, -1.6730, -1.4425,  1.1204,  0.3681, -0.3839,  1.1935, -1.7573,
         -0.8011,  0.6543,  0.3995,  0.1348, -2.3029,  0.3636, -0.0115,  0.2466,
          1.9492,  1.7609, -1.7673, -0.3518,  0.1305,  0.3662, -0.4011,  0.9839,
         -1.1147,  0.2526,  0.2427, -0.0984,  1.6934, -1.4659, -1.2089, -0.7998],
        [ 0.1231,  0.1081,  1.2979,  1.5194, -0.1330,  0.1057, -1.3819,  0.2168,
          0.3375,  0.2345,  0.9338, -0.6612,  0.3974,  1.0713,  0.3377, -0.1332,
          1.4359,  1.4971,  0.3869, -0.5180, -0.5155,  0.3949,  0.1650, -1.0674,
          0.5753,  0.9306, -1.3976,  0.7694, -0.1686,  0.4445, -0.0669, -0.6674],
        [-0.6367,  0.1544

In [210]:
(test_occupations := torch.randint(MIN_OCC, MAX_OCC, (MAX_LEN, BATCH_SIZE)).to(device))

tensor([[8, 5, 8,  ..., 2, 6, 9],
        [0, 1, 4,  ..., 9, 5, 0],
        [3, 1, 8,  ..., 7, 5, 9],
        ...,
        [3, 5, 0,  ..., 4, 1, 4],
        [7, 0, 3,  ..., 2, 2, 4],
        [5, 9, 0,  ..., 5, 6, 4]], device='cuda:0')

In [211]:
# seq, batch, embed
(logits := embedding(params=test_params, occupations=test_occupations))

tensor([[[-1.3968,  4.1818, -1.1994,  ...,  0.0000,  0.0000,  0.0000],
         [-0.8730,  2.6136, -0.7496,  ...,  0.0000,  0.0000,  0.0000],
         [-1.3968,  4.1818, -1.1994,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.3492,  1.0454, -0.2999,  ...,  0.0000,  0.0000,  0.0000],
         [-1.0476,  3.1363, -0.8996,  ...,  0.0000,  0.0000,  0.0000],
         [-1.5715,  4.7045, -1.3494,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.1746,  0.5227, -0.1499,  ...,  0.0000,  0.0000,  0.0000],
         [-0.6984,  2.0909, -0.5997,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.5715,  4.7045, -1.3494,  ...,  0.0000,  0.0000,  0.0000],
         [-0.8730,  2.6136, -0.7496,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.5238,  1.5682, -0.4498,  ...,  0.0000,  0.0000,  0.0000],
         [-0.1746,  0.5227, -0.1499,  ...,  0

In [212]:
print(logits.shape)

torch.Size([105, 32, 48])


In [213]:
import einops as ein
from torch import nn

In [214]:
embed_dims = 16
key_dims = 8
L = 100
batch_dims = 32

dtype = torch.complex64

# Bring embedding vectors into key space for dot products
w_k = nn.Parameter(torch.randn(embed_dims, key_dims, dtype=dtype))
w_q = nn.Parameter(torch.randn(embed_dims, key_dims, dtype=dtype))

# Reeinterpret embedding vectors as semantic adjustments
w_v = nn.Parameter(torch.randn(embed_dims, embed_dims, dtype=dtype))

In [215]:
logits = torch.randn(L, batch_dims, embed_dims, dtype=dtype)
logits

tensor([[[-0.6921+0.6522j,  0.5635+1.0998j,  0.8347-0.0733j,  ...,
          -0.3270+0.2469j, -0.0692-0.2164j,  0.6165-0.6430j],
         [ 1.3129-0.2031j,  0.4259-0.8563j, -0.5041-0.6327j,  ...,
           0.7089-0.2514j,  1.8550-0.4296j, -0.8086-0.0404j],
         [ 0.1761-0.8038j, -0.6270-0.3766j, -0.9874+1.9265j,  ...,
          -0.2693+0.3666j,  0.2185+0.0559j, -0.1989+0.3943j],
         ...,
         [-1.0189+0.4888j, -0.5706+0.0783j,  0.6153+0.4781j,  ...,
           0.0032-0.7725j, -0.2338+0.3859j,  0.6626-0.3513j],
         [-0.6344+0.7782j,  0.0228-0.4334j,  0.1028-0.7753j,  ...,
          -0.2902-0.1311j,  1.1067+1.0355j, -0.5448-0.9783j],
         [ 1.3785+0.1519j,  1.4912-0.4979j, -0.0743+0.1851j,  ...,
           0.3881+0.3145j,  0.6313-0.1174j,  0.1679-0.0291j]],

        [[ 0.0948+0.0432j, -0.2120+0.3042j, -0.1721+0.3746j,  ...,
           1.1749+0.6111j,  0.8997+0.8281j,  0.0615-0.9105j],
         [-0.1804+0.1474j, -0.0060+0.7942j, -0.1390-0.9689j,  ...,
           0.0

In [216]:
q = torch.matmul(logits, w_q)
k = torch.matmul(logits, w_k)
v = torch.matmul(logits, w_v)

_q = ein.rearrange(q, "s b k -> b s k")
_kH = ein.rearrange(k, "s b k -> b k s").conj()
_v = ein.rearrange(v, "s b k -> b s k")
similarities = _q @ _kH

# TODO: this causes a reallocation
similarities = torch.softmax(torch.abs(similarities), dim=-1).to(dtype=dtype)
adjustments = similarities @ _v
adjustments

tensor([[[ 1.4061e+00-1.7042e+00j, -2.3706e+00+2.1585e-01j,
          -4.5957e-01-4.4466e-01j,  ...,
          -3.7431e+00+1.0810e+00j, -1.2657e+00+2.7723e+00j,
          -1.0430e-01-5.3855e-01j],
         [ 6.6107e-01+1.8175e+00j, -1.6230e+00+7.4774e-02j,
           1.3100e+00+1.0855e+00j,  ...,
           1.6108e+00-3.0223e+00j, -7.1808e-01+7.9994e-01j,
           6.6655e-02+1.7232e+00j],
         [ 1.4068e+00-1.7035e+00j, -2.3695e+00+2.1618e-01j,
          -4.5895e-01-4.4513e-01j,  ...,
          -3.7410e+00+1.0803e+00j, -1.2662e+00+2.7721e+00j,
          -1.0487e-01-5.3756e-01j],
         ...,
         [ 6.6330e-01-1.9276e+00j, -2.2545e+00-1.6822e+00j,
           1.0419e+00-1.9083e+00j,  ...,
          -5.4447e+00+1.7153e+00j,  7.6035e-02+1.7367e+00j,
           1.1600e+00+1.8771e+00j],
         [-4.2091e+00+4.1052e+00j, -2.0763e+00+6.0765e-01j,
          -1.3160e+00+3.4147e-01j,  ...,
           5.2648e+00+6.3590e+00j,  2.9326e+00+2.4325e+00j,
           1.0283e+00+4.2742e-01j],
 

In [ ]:
def complex_attention_update(logits, w_q, w_k, w_v) -> torch.Tensor:
    """
    logits: (seq, batch, embed)
    w_q: (embed, key, head)
    w_k: (embed, key, head)
    w_v: (embed, embed, head)

    # TODO: rotary embedding?

    Returns: (seq, batch, embed) - The updated logits after passing through
    multi-headed attention
    """

    dtype = torch.complex64

    # TODO: justify that this is right. Translate to einops?
    # Weights are in the form (heads, embed, out) where out \in {key, embed}
    # (seq, batch, embed) -> (seq, 1, batch, embed)
    # (head, embed, out) ->  (1, head, embed, out)
    # (seq, 1, batch, embed) @ (1, head, embed, key) = (seq, head, batch, key)

    # Create a new "head" dimension
    logits = logits.unsqueeze(1)
    w_q = w_q.unsqueeze(0)
    w_k = w_k.unsqueeze(0)
    w_v = w_v.unsqueeze(0)

    q = torch.matmul(logits, w_q)
    k = torch.matmul(logits, w_k)
    v = torch.matmul(logits, w_v)

    # Batched matmul between A and B will
    # Broadcasting in matmul will only look at the batch dimensions
    # and NOT the sequence dimensions

    # note that vec-mat matmul is inherently batched.
    # Pytorch's version is simply an extension of the regular matmul "batching"

    _q = ein.rearrange(q, "s h b k -> b h s k")
    _kH = ein.rearrange(k, "s h b k -> b h k s").conj()
    _v = ein.rearrange(v, "s h b e -> b h s e")

    # (batch, head, seq, seq) @ (batch, head, sequence, embed)
    # TODO: them matrix dimensions should be clear from here.
    pattern = torch.softmax((_q @ _kH).abs(), dim=-1).to(dtype=dtype)
    updates = torch.sum(ein.rearrange(pattern @ _v, "b h s e -> s b e h"), dim=-1)
    return logits.squeeze(1) + updates

In [225]:
complex_attention_update(logits, w_q, w_k, w_v).shape

torch.Size([100, 32, 16])
torch.Size([100, 32, 16])


torch.Size([100, 32, 16])